# Modelling running power with the metrics provided by Stryd

In this notebook, we analyse the run recordings collected using Stryd power meter, and try to model the running power using the other metrics that Stryd provides. 

Stryd is the state-of-the-art closed-source running power meter, that is mounted on foot, and calculates/measures power using the built-in sensors (accelerometer, gyroscope, altimeter)

In [ ]:
from fitparse import FitFile
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

Import .fit file as pandas data frame

In [ ]:
def read_fit(path):
    fit = FitFile(path)
    
    def record_to_series(record):
        return pd.Series({f.name: f.value for f in record.fields})

    df = pd.DataFrame([record_to_series(record) for record in fit.get_messages("record")]).drop(["timestamp", "distance", "heart_rate", "enhanced_altitude", "enhanced_speed", "speed", "Form Power"], axis=1)
    return df

df = pd.concat([
    read_fit("../data/stryd/stryd-backgaden-1km.fit"),
    read_fit("../data/stryd/stryd-up-and-down.fit"),
    read_fit("../data/stryd/stryd-sport-field-circles.fit"),
    read_fit("../data/stryd/stryd-sport-field-lap.fit"),
    read_fit("../data/stryd/stryd-sport-field.fit"),
    read_fit("../data/stryd/stryd-backgaden-0.3km.fit"),
])

Absolute altitude and distance are not useful for our purposes. However, changes of altitude and distance over time might be important features. 

In [ ]:
df.loc[:,"altitude_diff"] = df.altitude.diff()
df.loc[:,"distance_diff"] = df.Distance.diff()
df.drop(["altitude", "Distance"], axis=1, inplace=True)
df = df.dropna()
df.shape

We remove all the data points that have zero or close to zero power, as they bare no information for our task (under the assumption that 0 W power is measured iff there's no motion)

In [ ]:
df = df[df.power > 5]
df.shape

# Correlation

We explore what features correlate to power -- our target variable

Observations:
* The strongest positive correlations to power can be seen for speed, followed by air power and cadence.
* Stance time shows strong negative correlation with leg spring stiffness, followed by power, cadence, vertical oscillation.


In [ ]:
sns.set_context(rc={"axes.labelsize":18})
sns.pairplot(df)

In [ ]:
sns.pairplot(df.loc[:, ["power", "cadence", "stance_time", "Speed", "Air Power", "Leg Spring Stiffness"]])

In [ ]:
sns.set_context("talk")
display(df.describe())
sns.heatmap(df.corr(), cmap="RdBu", center=0.0)
plt.show()

# Modelling
## Helper functions

For the purpose of our experiments we perform 5-fold cross validation, to get less biased performance metrics.

In [ ]:
def get_cv_stats(reg, X, y, name=""):
    cv = pd.DataFrame(cross_validate(reg, X, y, scoring=['r2', 'neg_mean_absolute_error']))
    return pd.Series({
        "mae_mean": cv.test_neg_mean_absolute_error.mean(),
        "mae_std": cv.test_neg_mean_absolute_error.std(),
        "r2_mean": cv.test_r2.mean(),
        "r2_std": cv.test_r2.std(),
    }, name=name)

def get_cv_mae(reg_class, alpha, X, y):
    reg = reg_class(alpha=alpha)
    return get_cv_stats(reg, X, y).mae_mean

def get_cv_r2(reg_class, alpha, X, y):
    reg = reg_class(alpha=alpha)
    return get_cv_stats(reg, X, y).r2_mean

## Preprocessing
* Separate the features (`X`) and the target (`y`)
* Some ML models benefit from features being uniformly scaled, hence Standard Scaler is applied (`X_scaled` and `y_scaled`)
* From physical models, we know that power is not related to speed linearly, but rather to a square of speed. Therefore, we can transform our features into combinations of polynomial terms (with degree up to 2) -- `X_polynomial`
* In addition, `X_selected` contains hand picked features. Cadence, vertical oscillation, stance time and speed are the features that can be approximated from accelerometer data, and should be relevant for power calculations.

In [ ]:
X = df.iloc[:,1:]
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)
X_selected = X_scaled.loc[:,["cadence", "vertical_oscillation", "stance_time", "Speed"]]

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_polynomial = poly_features.fit_transform(X_scaled)
feature_names = list(poly_features.get_feature_names())
for i, x in enumerate(X.columns):
    feature_names = [y.replace(f"x{i}", x) for y in feature_names]
X_polynomial = pd.DataFrame(X_polynomial, columns=feature_names)

y = df.iloc[:,0]
y_scaled = StandardScaler().fit_transform(y.values.reshape(-1,1)).reshape(-1,)


## Linear Regression
It's the simplest regression model, that fits `y=sum(w_i * x_i) + b` minimising sum of square difference between the predicted and the target values.

Note on the performance metrics:
* Negative mean absolute error (MAE) -- higher values (or lower absolute values) are better
* R^2 score -- coefficient of determination, a proportion of the target variance that can be predicted from the input features. Values closer to 1.0 are better

Fitting the regression with the basic set of features, gives average performance of R^2 = 0.606

In [ ]:
lr_naive_cv_summary = get_cv_stats(LinearRegression(), X_scaled, y, "Ordinary Least Squares")
lr_naive_cv_summary

Fitting with only the selected features makes the performance significantly worse, with R^2 of 0.385

In [ ]:
lr_manual_cv_summary = get_cv_stats(LinearRegression(), X_selected, y, "Ordinary Least Squares (Selected)")
lr_manual_cv_summary

Using the polynomial features, the performance gets even worse with R^2 of 0.256

In [ ]:
lr_poly_cv_summary = get_cv_stats(LinearRegression(), X_polynomial, y, "Ordinary Least Squares (Polynomial)")
lr_poly_cv_summary

## Ridge regression
Ridge regression can be used to attempt improving performance of the ordinary linear regression. In addition to minimising the square error, Ridge introduces a penalty for the size of the coefficients (L2 regularisation).

As opposed to the ordinary Linear Regression, Ridge requires a parameter for weight penalty (alpha). We try a range of different alpha values, and pick one that gives the best MAE on cross-validation.

The performance of Ridge with optimised alpha is slightly better than that of the original model, with R^2=0.619

In [ ]:
alpha = np.arange(0.1,100.0,0.1)
r2 = [get_cv_r2(Ridge, x, X_scaled, y) for x in alpha]
sns.lineplot(alpha, r2)
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.title("Original Features")
plt.show()
best_alpha = alpha[np.argmax(r2)]
best_alpha

In [ ]:
ridge_cv_summary = get_cv_stats(Ridge(best_alpha), X_scaled, y, "Ridge")
ridge_cv_summary

Fitting Ridge with polynomial features gives the best performance for alpha = 11.7, however R^2 is still lower than that of the plain linear model, even though the performance is significantly better if compared to the orignial model fitted with the polynomial features

In [ ]:
alpha = np.arange(0.1,20.0,0.1)
r2 = [get_cv_r2(Ridge, x, X_polynomial, y) for x in alpha]
sns.lineplot(alpha, r2)
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.title("Polynomial Features")
plt.show()
best_alpha = alpha[np.argmax(r2)]
best_alpha

In [ ]:
ridge_poly_cv_summary = get_cv_stats(Ridge(best_alpha), X_polynomial, y, "Ridge (Polynomial)")
ridge_poly_cv_summary

## Lasso Regression
Another improvement over the ordinary Linear Regression is Lasso. It employs L1 reguralisation, that penalises non-zero coefficients, effectively acting as feature selection.

Same as Ridge, it requires an alpha parameter.

Using the original features, we get the model, which is better than our original model, and shows R^2 of 0.638.

In [ ]:
alpha = np.arange(0.1,3.0,0.01)
r2 = [get_cv_r2(Lasso, x, X_scaled, y) for x in alpha]
sns.lineplot(alpha, r2)
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.title("Original Features")
plt.show()
best_alpha = alpha[np.argmax(r2)]
best_alpha

In [ ]:
lasso_cv_summary = get_cv_stats(Lasso(best_alpha), X_scaled, y, "Lasso")
lasso_cv_summary

Observing the coefficients, Lasso Regression model mainly makes use of speed, leg spring stiffness and change in altitude

In [ ]:
model = Lasso(best_alpha).fit(X_scaled, y)
coef = pd.Series(model.coef_, index=X_scaled.columns)
display("Weights",coef[coef.abs() > 0])

display("Intercept", model.intercept_)

Applying the same procedure to the polynomial features and alpha = 0.30, the performance is significantly better than that of the original model.

In [ ]:
alpha = np.arange(0.1,3.0,0.1)
r2 = [get_cv_r2(Lasso, x, X_polynomial, y) for x in alpha]
sns.lineplot(alpha, r2)
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.title("Polynomial Features")
plt.show()
best_alpha = alpha[np.argmax(r2)]
best_alpha

In [ ]:
lasso_poly_cv_summary = get_cv_stats(Lasso(best_alpha), X_polynomial, y, "Lasso (Polynomial)")
lasso_poly_cv_summary

The highest positive weights are given to speed and change in altitude. However, it is unexpected for the cadence to have a large negative weight

In [ ]:
model = Lasso(best_alpha).fit(X_polynomial, y)
coef = pd.Series(model.coef_, index=X_polynomial.columns)
display("Weights",coef[coef.abs() > 0])

display("Intercept", model.intercept_)

# Summary

L1 and L2 improves performance for both feature sets. And the best performing method is Lasso with the original features.

Surprisingly, feature engineering does not improve the results either. For example, even though it is expected that the power is related to the square of speed, rather than the speed itself, models does not seem to utilise it at all, bringing the performance down.

In [ ]:
pd.DataFrame([
    lr_naive_cv_summary, lr_manual_cv_summary, lr_poly_cv_summary,
    ridge_cv_summary, ridge_poly_cv_summary,
    lasso_cv_summary, lasso_poly_cv_summary,
])